## 0 AIS data process

In [1]:
# import libraries

import cudf as cd
import dask.dataframe as dd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statistics import mean
import math
from collections import Counter
import os
import matplotlib as mpl

In [ ]:
ais_data_path = r'file path.csv'

In [2]:
# Split datasets by months for easy computation

def data_splitter(rd_df_path, str_dfs_path, year):
    ais_data_year = dd.read_csv(rd_df_path, delimiter=';')
    ais_data_year['date_time_stamp'] = dd.to_datetime(ais_data_year[next(col for col in ais_data_year.columns if 'time' in col)], unit= 's')
    ais_data_year['month'] = ais_data_year['date_time_stamp'].dt.month
    months = ais_data_year['month'].unique().compute()

    for month in months:
        partition = ais_data_year[ais_data_year['month'] == month]
        partition = partition.compute()
        file_name = str_dfs_path + f'/data_{year}_{month:02d}.csv'
        
        partition.to_csv(file_name, index=False)
        print(month)
    
    return ais_data_year 


In [ ]:
ais_data = data_splitter(ais_data_path)

In [ ]:
## Vessel density plot - monthly

def monthly_unique_vessel_density(year_data):
    # get a list of unique months
    months = year_data['month'].unique()
    print(months)
    # initialize a dictionary to store the total number of unique vessels for each month
    total_unique_vessels_per_month = {month: 0 for month in months}
    # loop over each month and count the number of unique vessels
    for month in months:
        month_data = year_data[year_data['month'] == month]
        unique_vessels = month_data['mmsi'].nunique()
        total_unique_vessels_per_month[month] = unique_vessels
    # convert the dictionary to a DataFrame and sort by month
    results = pd.DataFrame.from_dict(total_unique_vessels_per_month, orient='index', columns=['unique_vessels'])
    results.index.name = 'month'
    results = results.sort_index()

    return results

    

In [ ]:
# Monthly unique vessel density _ dictionary

monthly_density_unique=  monthly_unique_vessel_density(ais_data)

In [ ]:
# create the DataFrame
monthly_density_unique.index = pd.to_datetime(monthly_density_unique.index, format='%m')
# create the plot
monthly_density_unique.plot()
# set the x-axis tick labels as month names and ylim for mmsi count range
plt.xticks(monthly_density_unique.index, monthly_density_unique.index.strftime('%b'))
plt.ylim(0, 700)
# add axis labels and title
plt.xlabel('Month')
plt.ylabel('Unique Vessels')
plt.title('Number of Unique Vessels per Month in YEAR')
# show the plot
plt.show()
# thought shows trends, month shall not be considered a feature. 

In [ ]:
# Time difference for all AIS messages.

def count_time_diff_dict(data_frame):
    # load AIS dataset into pandas DataFrame
    ais_time_diff_df = data_frame.copy()

    mmsi_tags = ais_time_diff_df['mmsi'].unique()
    net_ais_timediff_list = []

    for mmsi_unique in mmsi_tags:
        ais_df = ais_time_diff_df[ais_time_diff_df['mmsi'] == mmsi_unique]
        ais_df = ais_df.sort_values('date_time_stamp',ignore_index=True)
        ais_timediff = ais_df['date_time_stamp'].diff().dt.total_seconds()
        ais_timediff = ais_timediff.round(decimals = 2).dropna()
        ais_timediff = ais_timediff.drop(ais_timediff[ais_timediff.values > 900].index)
        ais_timediff_list = ais_timediff.values.tolist()
        net_ais_timediff_list.extend(ais_timediff_list)

    return net_ais_timediff_list


In [ ]:
# Average time difference for MMSIs. 

def count_avgtime_diff_dict_mmsi(data_frame):
    # Load AIS dataset into pandas DataFrame
    ais_time_diff_df = data_frame.copy()
    mmsi_tags = ais_time_diff_df['mmsi'].unique()
    net_avg_ais_timediff_list = []
    for mmsi_unique in mmsi_tags:
        ais_df = ais_time_diff_df[ais_time_diff_df['mmsi'] == mmsi_unique]
        ais_df = ais_df.sort_values('date_time_stamp',ignore_index=True)
        ais_timediff = ais_df['date_time_stamp'].diff().dt.total_seconds()
        ais_timediff = ais_timediff.round(decimals = 2).dropna()
        ais_timediff = ais_timediff.drop(ais_timediff[ais_timediff.values > 900].index)

        ais_timediff_list = ais_timediff.values.tolist()
        if len(ais_timediff_list) >= 1:
            avg_ais_timediff =  mean(ais_timediff_list)
            avg_ais_timediff = math.ceil(avg_ais_timediff)
        else:
            continue

        net_avg_ais_timediff_list.append(avg_ais_timediff)
    return net_avg_ais_timediff_list

In [ ]:
### import month dataset path

ais_data_month_path = r'C:\Users\Public\Documents\AIS Models\dataset\01_split_2020_v0_ais_dataset\data_2020_01.csv'
ais_month = 1
ais_data_month = pd.read_csv(ais_data_month_path)

In [ ]:
# For visualization (data exploration)
month_ais_timediff_list = count_time_diff_dict(ais_data_month)
net_ais_timediff_list = count_time_diff_dict(ais_data)
month_mmsi_count_ais_timediff_list = count_avgtime_diff_dict_mmsi(ais_data_month)
net_mmsi_count_ais_timediff_list = count_avgtime_diff_dict_mmsi(ais_data)

In [ ]:
# For visualization (data exploration)
# Count values in dictionaries

def count_time_diff_dict_sec(list_name): #in seconds
    list_ceil = [math.ceil(x) for x in list_name]
    list_ceil.sort()
    data = list_ceil
    count_seconds = Counter(data)
    return count_seconds

def count_time_diff_dict_min(list_name): #in minutes
    list_name.sort()
    minute = 60
    list_min = [(x / minute) for x in list_name]
    data = list_min
    count_min = Counter(data)
    return count_min


In [ ]:
# For visualization (data exploration)
def count_time_diff_dict_mmsi(list_name):
    list_name.sort()
    data = list_name
    count_mmsi_timediff = Counter(data)
    return count_mmsi_timediff

In [ ]:
# For visualization (data exploration)
ais_data_count_sec_dict = count_time_diff_dict_sec(net_ais_timediff_list)
ais_month_count_sec_dict = count_time_diff_dict_sec(month_ais_timediff_list)

ais_data_count_mmsi_time_dict = count_time_diff_dict_mmsi(net_mmsi_count_ais_timediff_list)
ais_month_count_mmsi_time_dict = count_time_diff_dict_mmsi(month_mmsi_count_ais_timediff_list)

In [ ]:
# For visualization (data exploration)
month_data_mmsi_count_120 = {k: v for k, v in ais_month_count_mmsi_time_dict.items() if k <= 120}
# checking for AIS interval in range exceeding 600sec (10min)
month_data_mmsi_count_range_600_900 = {k: v for k, v in ais_month_count_mmsi_time_dict.items() if k > 600}

In [ ]:
# For visualization (data exploration)
print('No. of vessels with average ais message time in range 600 to 900: ', sum(month_data_mmsi_count_range_600_900.values()))
plt.bar(*zip(*ais_month_count_mmsi_time_dict.items()))
plt.show()

In [ ]:
# For visualization (data exploration)
# create a pie chart from the dictionary values
labels = list(ais_month_count_mmsi_time_dict.keys())
sizes = list(ais_month_count_mmsi_time_dict.values())
plt.pie(sizes, labels=labels)

# set chart properties
plt.title('AIS messages by MMSI time difference')
plt.axis('equal')

# show chart
plt.show()

In [ ]:
# For visualization (data exploration)
# create a line chart from the dictionary values
x = list(ais_month_count_mmsi_time_dict.keys())
y = list(ais_month_count_mmsi_time_dict.values())
plt.plot(x, y)

# set chart properties
plt.title('January \'YEAR - AIS messages average interval plot')
plt.xlabel('Average time difference (seconds)')
plt.ylabel('MMSI/ vessel count')

# show chart
plt.show()

In [ ]:
# For visualization (data exploration)
new_month_data_120 = {k: v for k, v in ais_month_count_sec_dict.items() if k <= 120}
ais_data_count_min_dict = count_time_diff_dict_min(net_ais_timediff_list)
ais_month_count_min_dict = count_time_diff_dict_min(month_ais_timediff_list)

In [ ]:
# For visualization (data exploration)
def create_plot_timediff_count(dict_name_1,dict_name_2,Count_1, Count_2 ):
    df1 = pd.DataFrame.from_dict(dict_name_1, orient='index', columns=[Count_1])
    df2 = pd.DataFrame.from_dict(dict_name_2, orient='index', columns=[Count_2])
    df = pd.merge(df1, df2, how='outer', left_index=True, right_index=True)
    
    ax = df.plot(legend='AIS message')
    ax.set_xlabel('Time difference (minutes)')
    ax.set_ylabel('Net count')
    ax.set_title('AIS message - interval plot')
    
    plt.xticks(np.arange(0, 16, step=1))
    plt.tick_params(
    axis='x',          
    which='both',      
    bottom=True,      
    top=False,        
    labelbottom=True) 
    return ax


In [ ]:
# For visualization (data exploration)
def create_plot_timediff_mmsi_count(dict_name_1,dict_name_2 ):
    df1 = pd.DataFrame.from_dict(dict_name_1, orient='index', columns=['Year 2019 dataset '])
    df2 = pd.DataFrame.from_dict(dict_name_2, orient='index', columns=['January \'19 dataset'])
    df = pd.merge(df1, df2, how='outer', left_index=True, right_index=True)
    
    ax = df.plot(legend='AIS message')
    ax.set_xlabel('Average time difference (seconds)')
    ax.set_ylabel('MMSI/ Vessel count')
    ax.set_title('AIS message - average interval plot')
   
    plt.xticks(np.arange(0, 901, step=60))
    plt.tick_params(
    axis='x',          
    which='both',      
    bottom=True,      
    top=False,         
    labelbottom=True) 
    return ax


In [ ]:
# For visualization (data exploration)
with plt.style.context('ggplot'):
    create_plot_timediff_count(ais_data_count_min_dict,ais_month_count_min_dict, 'Year 2019 dataset ', 'January \'19 dataset')

with plt.style.context('ggplot'):
    create_plot_timediff_mmsi_count(ais_data_count_mmsi_time_dict, ais_month_count_mmsi_time_dict)

In [3]:
# For visualization (data exploration)
plt.bar(*zip(*new_month_data_120.items()))
plt.title('MONTH \'19 - AIS message: interval plot')
plt.xlabel('Time difference (seconds)')
plt.ylabel('Net count')
plt.xticks(np.arange(0, 121, step=20))
plt.show()

In [ ]:
# For visualization (data exploration)
# month_data_mmsi_count_120
plt.bar(*zip(*month_data_mmsi_count_120.items()))
plt.title('MONTH \'19 - AIS message: average interval plot')
plt.xlabel('Time difference (seconds)')
plt.ylabel('MMSI/ Vessel count')
plt.xticks(np.arange(0, 121, step=20))
plt.show()

In [ ]:
plt.bar(*zip(*new_data_120.items()))
plt.show()

In [ ]:
#MAIN OBJECTIVE# Toward efficient processing -> files have been stored by month

In [ ]:
# ## For visualization
# # PLOT - TRAJECTORY POINTS with COG INDICATOR (Gradient) - LON.LAT.COG

# def position_data_plot_simple(ais_data)  #, lon_lat_limits, map_image_path ):

#     mpl.rcParams['agg.path.chunksize']= 1000
#     # mpl.rcParams['path.simplify_threshold'] = 0.5

#     if not os.path.exists(r'W:\Neeraj_thesis\AIS_ship_trajectory_prediction\visualizations\p3_plots_cog_gradient'): 
#         os.mkdir(r'W:\Neeraj_thesis\AIS_ship_trajectory_prediction\\visualizations\p3_plots_cog_gradient')

#     # W:\Neeraj_thesis\AIS_ship_trajectory_prediction\visualizations

#     # Create list of unique mmsi_tags
#     mmsi_tags = ais_data['mmsi'].unique()

#     # Generate mmsi specific data for plots 
#     for mmsi_tag in mmsi_tags:
#         step_data = ais_data[ais_data['mmsi'] == int(mmsi_tag)]
#         step_data.sort_values('date_time_stamp')

#         # Course over ground series
#         step_data['cog'] = pd.to_numeric(step_data['cog'].str.replace(',', '.'), errors='coerce')
#         cog = step_data['cog']

#         map = Image.open(map_image_path)
#         fig, ax = plt.subplots(figsize=(map.width/100, map.height/100))

#         map_bounds = lon_lat_limits
#         BBox = (map_bounds['lon_min'], map_bounds['lon_max'], map_bounds['lat_min'], map_bounds['lat_max'] )
#         map_img = plt.imread(map_image_path)
        

#         im = ax.scatter( step_data['lon'], step_data['lat'], marker ='x', s = 0.5, c = step_data['cog'], cmap = 'twilight', vmin = 0 , vmax= 360 ) #step_data['cog'], cmap=colors) 

#         plt.xlabel('Longitude')
#         plt.ylabel( 'Latitude')
#         plt.title(f'Path of {mmsi_tag}')

#         # for index, row in step_data.iterrows():
#             # plt.text(row['lat'], row['lon'], row['date_time_stamp'], fontsize=8, ha='center')
#         # plt.text(data['lat'], data['long'], data['date_time_stamp']., fontsize=8)

#         ax.imshow(map_img, zorder=0, aspect='auto', extent= BBox) 
#         ax.set_xlim(BBox[0], BBox[1])
#         ax.set_ylim(BBox[2], BBox[3])
        
#         # Add a colorbar to show the color scale
#         c_bar = plt.colorbar(im)
#         c_bar.set_label(label = 'COG (Course Over Ground) \n 0/360\N{DEGREE SIGN}-> South to North; 180\N{DEGREE SIGN}-> North to South' , size = 13 )

#         plt.savefig(f'W:\\Neeraj_thesis\\AIS_ship_trajectory_prediction\\visualizations\\p3_plots_cog_gradient\\{mmsi_tag}.png',dpi =100)
#         plt.close()
#         plt.clf()

In [ ]:
# ## For visualization
# # PLOT - TRAJECTORY POINTS with COG INDICATOR (Gradient) - LON.LAT.COG

# def data_trajectory_plots_cog_color(ais_data, lon_lat_limits, map_image_path ):

#     mpl.rcParams['agg.path.chunksize']= 1000
#     # mpl.rcParams['path.simplify_threshold'] = 0.5

#     if not os.path.exists(r'W:\Neeraj_thesis\AIS_ship_trajectory_prediction\visualizations\p3_plots_cog_gradient'): 
#         os.mkdir(r'W:\Neeraj_thesis\AIS_ship_trajectory_prediction\\visualizations\p3_plots_cog_gradient')

#     # W:\Neeraj_thesis\AIS_ship_trajectory_prediction\visualizations

#     # Create list of unique mmsi_tags
#     mmsi_tags = ais_data['mmsi'].unique()

#     # Generate mmsi specific data for plots 
#     for mmsi_tag in mmsi_tags:
#         step_data = ais_data[ais_data['mmsi'] == int(mmsi_tag)]
#         step_data.sort_values('date_time_stamp')

#         # Course over ground series
#         step_data['cog'] = pd.to_numeric(step_data['cog'].str.replace(',', '.'), errors='coerce')
#         cog = step_data['cog']

#         map = Image.open(map_image_path)
#         fig, ax = plt.subplots(figsize=(map.width/100, map.height/100))

#         map_bounds = lon_lat_limits
#         BBox = (map_bounds['lon_min'], map_bounds['lon_max'], map_bounds['lat_min'], map_bounds['lat_max'] )
#         map_img = plt.imread(map_image_path)
        

#         im = ax.scatter( step_data['lon'], step_data['lat'], marker ='x', s = 0.5, c = step_data['cog'], cmap = 'twilight', vmin = 0 , vmax= 360 ) #step_data['cog'], cmap=colors) 

#         plt.xlabel('Longitude')
#         plt.ylabel( 'Latitude')
#         plt.title(f'Path of {mmsi_tag}')

#         # for index, row in step_data.iterrows():
#             # plt.text(row['lat'], row['lon'], row['date_time_stamp'], fontsize=8, ha='center')
#         # plt.text(data['lat'], data['long'], data['date_time_stamp']., fontsize=8)

#         ax.imshow(map_img, zorder=0, aspect='auto', extent= BBox) 
#         ax.set_xlim(BBox[0], BBox[1])
#         ax.set_ylim(BBox[2], BBox[3])
        
#         # Add a colorbar to show the color scale
#         c_bar = plt.colorbar(im)
#         c_bar.set_label(label = 'COG (Course Over Ground) \n 0/360\N{DEGREE SIGN}-> South to North; 180\N{DEGREE SIGN}-> North to South' , size = 13 )

#         plt.savefig(f'W:\\Neeraj_thesis\\AIS_ship_trajectory_prediction\\visualizations\\p3_plots_cog_gradient\\{mmsi_tag}.png',dpi =100)
#         plt.close()
#         plt.clf()

In [ ]:
# ## For visualization
# # PLOT - TRAJECTORY POINTS with COG INDICATOR (Gradient) - LON.LAT.COG , Vesesl Navigation status as the sorting crriteria 

# #--> Need to make a comparission between features (eg. navigation status vs lengt (vessel type): Length not provided a feature)

# def data_trajectory_plots_var_nav_status(ais_data, lon_lat_limits, map_image_path ):

#     mpl.rcParams['agg.path.chunksize']= 1000
#     # mpl.rcParams['path.simplify_threshold'] = 0.5

#     if not os.path.exists(f'W:\\Neeraj_thesis\\AIS_ship_trajectory_prediction\\visualizations\\p8_plots_nav_status'): 
#         os.mkdir(f'W:\\Neeraj_thesis\\AIS_ship_trajectory_prediction\\visualizations\\p8_plots_nav_status')



#     for nav_status in set(ais_data.nav_status):

#         map = Image.open(map_image_path)
#         fig, ax = plt.subplots(figsize=(map.width/100, map.height/100))

#         # Filter data for vessels with length less than the threshold
#         ais_data_nav_status = ais_data[ais_data['nav_status'] == nav_status]

#         # Plot data for vessels with length less than the threshold
#         # cog_data_nav_status = ais_data_nav_status['cog']
#         map = Image.open(map_image_path)
#         fig, ax = plt.subplots(figsize=(map.width/100, map.height/100))

#         map_bounds = lon_lat_limits
#         BBox = (map_bounds['lon_min'], map_bounds['lon_max'], map_bounds['lat_min'], map_bounds['lat_max'] )
#         map_img = plt.imread(map_image_path)
        

#         im = ax.scatter( ais_data_nav_status['lon'], ais_data_nav_status['lat'], marker ='x', s=0.4 , c = ais_data_nav_status['mmsi'], cmap = 'twilight' , vmin = int(ais_data_nav_status['mmsi'].min()) , vmax = int(ais_data_nav_status['mmsi'].max())) #step_data['cog'], cmap=colors) 

#         plt.xlabel('Longitude')
#         plt.ylabel( 'Latitude')
#         plt.title(f'Path of Ships with nav_status: {nav_status}')
#         ax.imshow(map_img, zorder=0, aspect='auto', extent= BBox) 
#         ax.set_xlim(BBox[0], BBox[1])
#         ax.set_ylim(BBox[2], BBox[3])
        
#         # Add a colorbar to show the color scale
#         c_bar = plt.colorbar(im)
#         c_bar.set_label(label = 'mmsi' , size = 13 )

#         plt.savefig(f'W:\\Neeraj_thesis\\AIS_ship_trajectory_prediction\\visualizations\\p8_plots_nav_status\\nav_status_is_{nav_status}.png',dpi =100)
# #   f'W:\\Neeraj_thesis\\AIS_ship_trajectory_prediction\\visualizations\\p3_plots_cog_gradient\\{mmsi_tag}.png',dpi =100

#         plt.close()
#         plt.clf()

    

In [ ]:
# ## For visualization
# data_trajectory_plots_cog_color(
# ais_data = ais_data_clean_5, 
# lon_lat_limits = map_limits, 
# map_image_path ='W:\\Neeraj_thesis\\AIS_ship_trajectory_prediction\\visualizations\\maps\\map_one.png'
# )

# ## Generating map
# #   // Define the LatLng coordinates for the polygon's path.
  
# #     /* MH max longitude: 11.445
# #       MH min longitude: 9.4
# #       MH max latitude: 59.931
# #       MH min latitude: 58.6
# #      */
# #   const triangleCoords = [
# #     { lat: 59.931, lng: 9.4 },
# #     { lat: 58.6, lng: 9.4 },
# #     { lat: 58.6, lng: 11.445 },
# #     { lat: 59.931, lng: 11.445 },
# #   ];

In [ ]:
# ## For visualization
# data_trajectory_plots_var_nav_status(
# ais_data = ais_data_oslo_2019, 
# lon_lat_limits = map_limits_2019, 
# map_image_path ='W:\\Neeraj_thesis\\AIS_ship_trajectory_prediction\\visualizations\\maps\\oslo_region_map_zoom.png'
# )

In [ ]:
# ## For visualization 
# ## VESSEL DENSITY PLOT_TIME REFERENCE_time in a day_24hrs

# d_first = 1
# d_last =31
# hr_first = 0
# hr_last = 23

# days = list(range(d_first,d_last+1))
# # print(days)

# hours = list(range(hr_first,hr_last+1))
# # print(hours)

# # initialize a dictionary to store the total number of vessels for each hour
# total_vessels_per_hour = {hour: 0 for hour in hours}

# # initialize a dictionary to store the number of days for each hour
# n_days_per_hour = {hour: 0 for hour in hours}

# # vessel_density_df = pd.DataFrame
# for day  in days:
#     step_data_day = ais_data_mh_202201[ais_data_mh_202201['date_time_stamp'].dt.day == day]
#     # print(step_data_day['date_time_stamp'].dt.day.unique())

#     for hour in hours:
#         step_data_hour = step_data_day[(step_data_day['date_time_stamp'].dt.hour == hour)]
#         step_data_hour.sort_values('date_time_stamp')
#         n_vessels_hour = int(step_data_hour['mmsi'].nunique())
#         # print(day, hour, n_vessels_hour)

#         # increment the total number of vessels for this hour
#         total_vessels_per_hour[hour] += n_vessels_hour

#         # increment the number of days for this hour
#         n_days_per_hour[hour] += 1

# # calculate the average number of vessels per hour
# avg_vessels_per_hour = {hour: total_vessels_per_hour[hour] / n_days_per_hour[hour] for hour in hours}

# print(avg_vessels_per_hour)

# lists = sorted(avg_vessels_per_hour.items()) # sorted by key, return a list of tuples

# x, y = zip(*lists) # unpack a list of pairs into two tuples

# plt.plot(x, y)
# plt.show()

# ### Thought not a proper representation - probably due to MISSING datapoint:
# # However with an correct data -- time trends shall be related to vessel density and plot types.